In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import cv2
import PIL.Image as Image
import os
import pathlib


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [3]:
dataset_url= "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir= tf.keras.utils.get_file('flower_photos', origin= dataset_url, cache_dir= '.', untar=True)

In [4]:
data_dir

'.\\datasets\\flower_photos'

In [5]:
data_dir= pathlib.Path(data_dir)
data_dir

WindowsPath('datasets/flower_photos')

In [6]:
list(data_dir.glob("*/*.jpg"))[:5]

[WindowsPath('datasets/flower_photos/daisy/100080576_f52e8ee070_n.jpg'),
 WindowsPath('datasets/flower_photos/daisy/10140303196_b88d3d6cec.jpg'),
 WindowsPath('datasets/flower_photos/daisy/10172379554_b296050f82_n.jpg'),
 WindowsPath('datasets/flower_photos/daisy/10172567486_2748826a8b.jpg'),
 WindowsPath('datasets/flower_photos/daisy/10172636503_21bededa75_n.jpg')]

In [7]:
image_count= len(list(data_dir.glob("*/*.jpg")))
image_count

3670

In [8]:
roses= list(data_dir.glob("roses/*"))
roses[:5]

[WindowsPath('datasets/flower_photos/roses/10090824183_d02c613f10_m.jpg'),
 WindowsPath('datasets/flower_photos/roses/102501987_3cdb8e5394_n.jpg'),
 WindowsPath('datasets/flower_photos/roses/10503217854_e66a804309.jpg'),
 WindowsPath('datasets/flower_photos/roses/10894627425_ec76bbc757_n.jpg'),
 WindowsPath('datasets/flower_photos/roses/110472418_87b6a3aa98_m.jpg')]

In [9]:
flower_images_dict= {
    "roses": list(data_dir.glob("roses/*")),
    "daisy": list(data_dir.glob("daisy/*")),
    "dandelion": list(data_dir.glob("dandelion/*")),
    "tulips": list(data_dir.glob("tulips/*")),
    "sunflowers": list(data_dir.glob("sunflowers/*"))
}

In [10]:
flower_labels_dict= {
    "roses": 0,
    "daisy": 1,
    "dandelion": 2,
    "tulips": 3,
    "sunflowers": 4
}

In [11]:
len(flower_images_dict["roses"])

641

In [12]:
img= cv2.imread(str(flower_images_dict["roses"][0]))
img.shape

(240, 179, 3)

In [14]:
cv2.resize(img, (180, 180)).shape

(180, 180, 3)

In [15]:
X, y= [], []

for flowers, images in flower_images_dict.items():
  for image in images:
    img= cv2.imread(str(image))
    resized_img= cv2.resize(img, (180, 180))
    X.append(resized_img)
    y.append(flower_labels_dict[flowers])

In [16]:
X = np.array(X)
y= np.array(y)

In [17]:
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=0)

In [18]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

MemoryError: Unable to allocate 1.99 GiB for an array with shape (2752, 180, 180, 3) and data type float64

In [19]:
num_classes= 5
model= Sequential([
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])
model.compile(optimizer='adam',
              loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics= ['acc'])

model.fit(X_train_scaled, y_train, epochs=25)

NameError: name 'X_train_scaled' is not defined

In [ ]:
model.evaluate(X_test_scaled, y_test)

In [ ]:
data_augmentation= keras.Sequential([
    layers.experimental.preprocessing.RandomRotation(0.5)
])

In [ ]:
plt.axis('off')
plt.imshow(X[7])

In [ ]:
plt.axis('off')
plt.imshow(data_augmentation(X)[0].numpy().astype('uint8'))

In [ ]:
data_augmentation= keras.Sequential([
    layers.experimental.preprocessing.RandomZoom(0.5)
])

In [ ]:
plt.axis('off')
plt.imshow(data_augmentation(X)[7].numpy().astype('uint8'))

In [ ]:
data_augmentation= keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal",
                                                 input_shape=(img_height,
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1)
])